In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.1/StatisticalRethinking/zZGTK.ji for StatisticalRethinking [2d09df54-9d0f-5258-8220-54c2a3d4fbee]
└ @ Base loading.jl:1184
loaded

File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.034, 0.033, 0.033, 0.033) seconds, 0.13 seconds total
Sampling took (0.067, 0.048, 0.050, 0.057) seconds, 0.22 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.3e-02    0.99  -894  -891  -891  1.9e+03  8.4e+03  1.0e+00
accept_stat__   0.92  1.5e-03    0.10  0.71  0.95   1.0  4.7e+03  2.1e+04  1.0e+00
stepsize__      0.83  3.3e-02   0.047  0.78  0.88  0.88  2.0e+00  9.0e+00  2.6e+13
treedepth__      2.0  7.3e-02    0.74   1.0   2.0   3.0  1.0e+02  4.7e+02  1.0e+00
n_le

end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*